In [10]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
trn = pd.read_csv('./data/train.csv')

In [12]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [13]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

## 결측치 처리

- Origin_Airport, Origin_Airport_ID : 1:1 대응 확인
- Airport, States 관계 확인
- 결측 대체

In [14]:
# ## 출발 공항, 공항번호 1:1 대응 확인
# origin_airports = trn['Origin_Airport'].unique().tolist()

# check = []
# for airport in origin_airports:
#     id_list = trn[trn['Origin_Airport']==airport]['Origin_Airport_ID'].unique().tolist()
#     if (len(id_list) != 1):
#         check.append(airport)
# print(check)

# ## 도착 공항, 공항번호 1:1 대응 확인
# depart_airports = trn['Destination_Airport'].unique().tolist()
# check = []
# for airport in depart_airports:
#     id_list = trn[trn['Destination_Airport']==airport]['Destination_Airport_ID'].unique().tolist()
#     if (len(id_list) != 1):
#         check.append(airport)
# print(check)

In [15]:
## origin state - airport dictionary 생성
temp = trn[~trn['Origin_State'].isnull()]
states = temp['Origin_State'].unique().tolist()
print(len(states))

origin_state_airpot_dict = dict()
for stat in states:
    airports = temp[temp['Origin_State']==stat]['Origin_Airport'].unique().tolist()
    for air in airports:
        origin_state_airpot_dict[air] = stat
        

temp = trn[~trn['Destination_State'].isnull()]
destination_states = temp['Destination_State'].unique().tolist()
print(len(destination_states))

destination_state_airpot_dict = dict()
for stat in destination_states:
    airports = temp[temp['Destination_State']==stat]['Destination_Airport'].unique().tolist()
    for air in airports:
        destination_state_airpot_dict[air] = stat

## ohio yng 공항은 데이터 내에서 찾을 수 없음 -> 검색정보 수기로 채우기
destination_state_airpot_dict['YNG'] = 'Ohio'

52
52


- dictionary 정보로 states 결측 대체
    - origin_state_airpot_dict
    - destination_states

In [16]:
def isna(x):
    return x!=x

In [17]:
trn['Origin_State'] = trn.apply(lambda x:origin_state_airpot_dict[x['Origin_Airport']] if(isna(x['Origin_State'])==True) else x['Origin_State'], axis=1)
trn['Destination_State'] = trn.apply(lambda x:destination_state_airpot_dict[x['Destination_Airport']] if(isna(x['Destination_State'])==True) else x['Destination_State'], axis=1)

In [18]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

=> State 결측 제거 완료

- Airline 정보확인

In [19]:
## Airline, Carrier_ID(DOT) 1:1 대응 확인
temp = trn[~trn['Airline'].isnull()]
temp['Carrier_ID(DOT)'] = temp['Carrier_ID(DOT)'].astype(str)
airlines = temp.Airline.unique().tolist()

dot_dict = dict()
check = []
cnt=0
for air in airlines:
    dot = temp[temp['Airline']==air]['Carrier_ID(DOT)'].unique().tolist()    
    dot = [x for x in dot if x != 'nan']
    cnt+=1
    if (len(dot) != 1):
        check.append(air)
        break
    dot_dict[dot[0]] = air
print(check)



## IATA 정보 airline과 1:1 대응인 지점 확인
temp = trn[~trn['Carrier_Code(IATA)'].isnull()]
iata_ids = temp['Carrier_Code(IATA)'].unique().tolist()

single_iata_dict = dict()
for iata in iata_ids:
    airline = temp[temp['Carrier_Code(IATA)']==iata].Airline.unique()
    airline = [x for x in airline if isna(x) == False]
    if(len(airline)==1):
        single_iata_dict[iata] = airline[0]
        
        
## Airline,tail_number 확인
temp = trn[~trn['Airline'].isnull()]
airlines = temp.Airline.unique().tolist()
airline_tail_dict = dict()

for air in airlines:
    tail_list = temp[temp['Airline']==air]['Tail_Number'].unique().tolist()
    airline_tail_dict[air] = tail_list

C:\Users\ppang\AppData\Local\Temp\ipykernel_15312\2898317092.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Carrier_ID(DOT)'] = temp['Carrier_ID(DOT)'].astype(str)


[]


- airline 결측 대체
    - dot dictionary : airline과 1:1 대응
    - single_iata_dict : 일부 iata정보 airline과 1:1 대응
    - airline_tail_dict : 일부 airline과 tail_number 1:1 대응

In [28]:
trn['Carrier_ID(DOT)'] = trn['Carrier_ID(DOT)'].astype(str)
for i in range(len(trn)):
    if(isna(trn.loc[i, 'Airline']) == True):
        if(trn.loc[i, 'Carrier_ID(DOT)'] != 'nan'):
            trn.loc[i, 'Airline'] = dot_dict[trn.loc[i, 'Carrier_ID(DOT)']]
            
        elif(trn.loc[i, 'Carrier_Code(IATA)'] in single_iata_dict):
            trn.loc[i, 'Airline'] = single_iata_dict[trn.loc[i, 'Carrier_Code(IATA)']]
            
        single_tail_check = []
        for air in airlines:
            if (trn.loc[i, 'Tail_Number'] in airline_tail_dict[air]):
                single_tail_check.append(air)
        ## 1:1 대응의 경우 airline 대체
        if (len(single_tail_check) == 1):
            trn.loc[i, 'Airline'] = single_tail_check[0]

In [29]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                        315
Carrier_Code(IATA)          108990
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
dtype: int64

=> 항공사 정보가 중복되는 315건 => 동일한 tail number에서 빈도수 많은 값으로 대체

In [38]:
for i in range(len(trn)):
    if(isna(trn.loc[i, 'Airline']) == True):
        try:
            trn.loc[i, 'Airline'] = list(dict(trn[trn['Tail_Number']==trn.loc[i, 'Tail_Number']]['Airline'].value_counts()[:1]).keys())[0]
        except:
            pass

In [45]:
# 남은 2개는 정황상 Delta Air Lines Inc.(같은 항공편 조회)
trn['Airline'] = trn['Airline'].fillna('Delta Air Lines Inc.')

In [46]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)          108990
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
dtype: int64

- 필요없는 column drop
    - ID : 식별자
    - Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
    - Destination_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
    - Cancelled : 전부 0
    - Diverted : 전부 0
    - Carrier_Code(IATA), Carrier_ID(DOT) : 항공사 정보로 통일

In [48]:
trn.to_csv('./data/temp_trn.csv',index=False)

In [ ]:
trn = trn.drop(columns=['ID','Origin_Airport_ID','Destination_Airport_ID','Cancelled','Diverted','Carrier_Code(IATA)','Carrier_ID(DOT)'])

In [101]:
trn = pd.read_csv('./data/temp_trn.csv')

- 출발 시간, 착륙시간 확인

In [102]:
## 시간, 분 분리
trn['Estimated_Departure_Hour'] = trn['Estimated_Departure_Time'].apply(lambda x:x//100)
trn['Estimated_Departure_Min'] = trn['Estimated_Departure_Time'].apply(lambda x:x%100)
trn['Estimated_Arrival_Hour'] = trn['Estimated_Arrival_Time'].apply(lambda x:x//100)
trn['Estimated_Arrival_Min'] = trn['Estimated_Arrival_Time'].apply(lambda x:x%100)

In [103]:
## 항공편별 출발, 도착 시간 평균
matching_df = round(trn.groupby(['Origin_Airport','Destination_Airport'])['Estimated_Departure_Hour','Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min'].mean().reset_index())
matching_df['identity'] = matching_df['Origin_Airport']+matching_df['Destination_Airport']
matching_df.head(3)

C:\Users\ppang\AppData\Local\Temp\ipykernel_15312\968825694.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  matching_df = round(trn.groupby(['Origin_Airport','Destination_Airport'])['Estimated_Departure_Hour','Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min'].mean().reset_index())


,Origin_Airport,Destination_Airport,Estimated_Departure_Hour,Estimated_Departure_Min,Estimated_Arrival_Hour,Estimated_Arrival_Min,identity
0,ABE,ATL,12.0,29.0,14.0,16.0,ABEATL
1,ABE,BNA,17.0,37.0,18.0,33.0,ABEBNA
2,ABE,CLT,11.0,30.0,13.0,34.0,ABECLT


In [104]:
for i in range(len(trn)):
    identity = trn.loc[i,'Origin_Airport']+trn.loc[i,'Destination_Airport']
    if ((isna(trn.loc[i,'Estimated_Departure_Time'])) & isna(trn.loc[i,'Estimated_Arrival_Time'])):
        if(identity in matching_df['identity'].tolist()):
            trn.iloc[i,-4:] = matching_df[matching_df['identity']==identity].iloc[:,2:-1]
            
        else:
            trn.loc[i,'Estimated_Departure_Hour'] = round(trn.loc[i,'Estimated_Departure_Hour'].mean())
            trn.loc[i,'Estimated_Departure_Min'] = round(trn.loc[i,'Estimated_Departure_Min'].mean())
            trn.loc[i,'Estimated_Arrival_Hour'] = round(trn.loc[i,'Estimated_Arrival_Hour'].mean())
            trn.loc[i,'Estimated_Arrival_Min'] = round(trn.loc[i,'Estimated_Arrival_Min'].mean())
    
    elif(isna(trn.loc[i,'Estimated_Departure_Time'])):
        if(identity in matching_df['identity'].tolist()):
            trn.iloc[i,-4:-2] = matching_df[matching_df['identity']==identity].iloc[:,2:-3]
        else:
            trn.loc[i,'Estimated_Departure_Hour'] = round(trn.loc[i,'Estimated_Departure_Hour'].mean())
            trn.loc[i,'Estimated_Departure_Min'] = round(trn.loc[i,'Estimated_Departure_Min'].mean())
            
    elif(isna(trn.loc[i,'Estimated_Arrival_Time'])):
        if(identity in matching_df['identity'].tolist()):
            trn.iloc[i,-2:] = matching_df[matching_df['identity']==identity].iloc[:,-3:-1]
        else:
            trn.loc[i,'Estimated_Arrival_Hour'] = round(trn.loc[i,'Estimated_Arrival_Hour'].mean())
            trn.loc[i,'Estimated_Arrival_Min'] = round(trn.loc[i,'Estimated_Arrival_Min'].mean())

In [105]:
trn.isnull().sum()

Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Origin_Airport                   0
Origin_State                     0
Destination_Airport              0
Destination_State                0
Distance                         0
Airline                          0
Tail_Number                      0
Delay                       744999
Estimated_Departure_Hour        20
Estimated_Departure_Min         20
Estimated_Arrival_Hour          14
Estimated_Arrival_Min           14
dtype: int64

In [107]:
trn[trn['Estimated_Departure_Hour'].isnull()]

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport,Origin_State,Destination_Airport,Destination_State,Distance,Airline,Tail_Number,Delay,Estimated_Departure_Hour,Estimated_Departure_Min,Estimated_Arrival_Hour,Estimated_Arrival_Min
16805,3,31,NaN,10.0,MDW,Illinois,PBI,Florida,1128.0,Southwest Airlines Co.,N8545V,NaN,NaN,NaN,0.0,10.0
58077,1,5,NaN,1256.0,EGE,Colorado,SLC,Utah,279.0,SkyWest Airlines Inc.,N763SK,NaN,NaN,NaN,12.0,56.0
118877,8,29,NaN,920.0,DEN,Colorado,BMI,Illinois,834.0,Frontier Airlines Inc.,N304FR,NaN,NaN,NaN,9.0,20.0
143069,7,9,NaN,1244.0,GJT,Colorado,ORD,Illinois,1100.0,SkyWest Airlines Inc.,N791SK,NaN,NaN,NaN,12.0,44.0
227313,12,28,NaN,NaN,FLL,Florida,BHM,Alabama,650.0,Southwest Airlines Co.,N561WN,Not_Delayed,NaN,NaN,NaN,NaN
332376,6,30,NaN,914.0,VPS,Florida,HTS,West Virginia,589.0,Allegiant Air,257NV,NaN,NaN,NaN,9.0,14.0
348173,12,21,NaN,1925.0,BNA,Tennessee,OKC,Oklahoma,616.0,Southwest Airlines Co.,N215WN,NaN,NaN,NaN,19.0,25.0
547605,8,14,NaN,1332.0,MDT,Pennsylvania,SRQ,Florida,942.0,Allegiant Air,232NV,NaN,NaN,NaN,13.0,32.0
570081,12,21,NaN,NaN,DTW,Michigan,BZN,Montana,1400.0,Delta Air Lines Inc.,N338NB,Not_Delayed,NaN,NaN,NaN,NaN
639953,9,29,NaN,921.0,DEN,Colorado,BMI,Illinois,834.0,Frontier Airlines Inc.,N943FR,NaN,NaN,NaN,9.0,21.0


In [ ]:
# trn = trn.drop(columns=['ID','Origin_Airport_ID','Destination_Airport_ID','Cancelled','Diverted','Carrier_Code(IATA)','Carrier_ID(DOT)','Estimated_Departure_Time','Estimated_Arrival_Time'])
trn = trn.drop(columns=['Estimated_Departure_Time','Estimated_Arrival_Time'])

In [ ]:
trn.to_csv('./data/temp_trn.csv',index=False)